<a href="https://colab.research.google.com/github/saad1ibn2akhter/AI_Olympiad_problems_and_solutions/blob/main/pytorch_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import os
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download =True,
    transform=ToTensor(),
)

train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download = True,
    transform = ToTensor(),
)



In [36]:
batch_size = 64
training_data = DataLoader(train_data , batch_size = batch_size)
testing_data = DataLoader(test_data , batch_size = batch_size)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [37]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits





In [38]:
#BASIC trial
model = NeuralNetwork().to(device)
print(model)

X = torch.rand(1,28,28,device=device)
logits = model(X)
pred_a = nn.Softmax(dim=1)(logits)
y_pred = pred_a.argmax(1)
print(f"the image is classified as [{y_pred}]")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
the image is classified as [tensor([1])]


In [39]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


In [40]:
def train(dataloader , model , loss , optimizer):
  size = len(dataloader.dataset )
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device) , y.to(device)
    pred = model(X)
    l = loss(pred,y)

    l.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      l, current = l.item(), (batch + 1) * len(X)
      print(f"loss: {l:>7f}  [{current:>5d}/{size:>5d}]")


In [41]:
def test(dataloader , model , loss):
  size = len(dataloader.dataset )
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for X ,y in dataloader:
      X,y = X.to(device),y.to(device)
      pred = model(X)
      test_loss+=loss(pred,y).item()
      # correct +=(pred.argmax(1)==y).type(torch.float).sum().item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss/=num_batches
  correct/=size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [42]:
epochs = 5
for t in range(epochs):
  print(f"Epoch : {t+1}")
  train(training_data , model , loss, optimizer)
  test(testing_data , model , loss)
print("Done")


Epoch : 1
loss: 2.324047  [   64/60000]
loss: 2.307196  [ 6464/60000]
loss: 2.283304  [12864/60000]
loss: 2.271506  [19264/60000]
loss: 2.249992  [25664/60000]
loss: 2.225188  [32064/60000]
loss: 2.227742  [38464/60000]
loss: 2.197439  [44864/60000]
loss: 2.204325  [51264/60000]
loss: 2.163018  [57664/60000]
Test Error: 
 Accuracy: 44.7%, Avg loss: 2.157403 

Epoch : 2
loss: 2.177696  [   64/60000]
loss: 2.162578  [ 6464/60000]
loss: 2.104975  [12864/60000]
loss: 2.121132  [19264/60000]
loss: 2.062742  [25664/60000]
loss: 2.004633  [32064/60000]
loss: 2.030448  [38464/60000]
loss: 1.954700  [44864/60000]
loss: 1.972243  [51264/60000]
loss: 1.890132  [57664/60000]
Test Error: 
 Accuracy: 58.7%, Avg loss: 1.891352 

Epoch : 3
loss: 1.929968  [   64/60000]
loss: 1.893319  [ 6464/60000]
loss: 1.785968  [12864/60000]
loss: 1.828143  [19264/60000]
loss: 1.702134  [25664/60000]
loss: 1.663635  [32064/60000]
loss: 1.679142  [38464/60000]
loss: 1.586745  [44864/60000]
loss: 1.618172  [51264/600